In [1]:
import pysam
import numpy as np
from tqdm import tqdm

samfile1 = pysam.AlignmentFile("/Volumes/Extreme SSD/HP/cap1_m6am_test.bam", "rb")
samfile2 = pysam.AlignmentFile("/Volumes/Extreme SSD/HP/cap2_m6am_test.bam", "rb")

In [2]:
# Fetching all reads
reads1 = []
reads2 = []

for i in range(24):
    reads1.append([])
    reads2.append([])
    
i = 1
for i in range(1,25):
    if i==23:
        for read in samfile2.fetch('chrX'):
            reads2[i-1].append(read) 
        for read in samfile1.fetch('chrX'):
            reads1[i-1].append(read) 
    if i==24:
        for read in samfile2.fetch('chrY'):
            reads2[i-1].append(read) 
        for read in samfile1.fetch('chrY'):
            reads1[i-1].append(read) 
    elif i!=23 and i!=24:
        for read in samfile1.fetch(f'chr{i}'):
            reads1[i-1].append(read)
        for read in samfile2.fetch(f'chr{i}'):
            reads2[i-1].append(read)
    i+=1

In [3]:
from pyfaidx import Fasta
file_path = "/Volumes/Extreme SSD/HP/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna"
fasta = Fasta(file_path)

In [4]:
# Return the base at specified chromosome and position 
from ipynb.fs.full. Important_Functions import *

In [11]:
# Separates reads into m6Am, Am, and other
# For fwd reads
def add_read_info_fwd(read, pos_count_array_m6Am, pos_count_array_Am, pos_count_array):                    
    if read.query_sequence[0] == 'A':
        if i == 22:
            pos_count_array_m6Am.append("chrX")
        if i == 23:
            pos_count_array_m6Am.append("chrY")
        else:
            pos_count_array_m6Am.append(f'chr{i+1}')
        pos_count_array_m6Am.append(read.reference_end-40)
        pos_count_array_m6Am.append(read.reference_end-39)
        pos_count_array_m6Am.append("+")
        pos_count_array_m6Am.append(str(read.query_name).split("_")[-1] + read.query_sequence)   
        pos_count_array_m6Am.append(str(read.query_sequence))
    elif read.query_sequence[0] == 'G' and chr_loc(f'chr{i+1}', read.reference_end-39, fasta) == 'A':
        if i == 22:
            pos_count_array_Am.append("chrX")
        if i == 23:
            pos_count_array_Am.append("chrY")
        else:
            pos_count_array_Am.append(f'chr{i+1}')
        pos_count_array_Am.append(read.reference_end-40)
        pos_count_array_Am.append(read.reference_end-39)
        pos_count_array_Am.append("+")
        pos_count_array_Am.append(str(read.query_name).split("_")[-1] + read.query_sequence)
        pos_count_array_Am.append(str(read.query_sequence))
    else:
        if i == 22:
            pos_count_array.append("chrX")
        if i == 23:
            pos_count_array.append("chrY")
        else:
            pos_count_array.append(f'chr{i+1}')
        pos_count_array.append(read.reference_end-40)
        pos_count_array.append(read.reference_end-39)
        pos_count_array.append("+")
        pos_count_array.append(str(read.query_name).split("_")[-1] + read.query_sequence) 
        pos_count_array.append(str(read.query_sequence))

In [12]:
# For rev reads
def add_read_info_rev(read, pos_count_array_m6Am, pos_count_array_Am, pos_count_array):
    if read.query_sequence[0] == 'T':
        if i == 22:
            pos_count_array_m6Am.append("chrX")
        if i == 23:
            pos_count_array_m6Am.append("chrY")
        else:
            pos_count_array_m6Am.append(f'chr{i+1}')
        pos_count_array_m6Am.append(read.reference_end-1)
        pos_count_array_m6Am.append(read.reference_end)
        pos_count_array_m6Am.append("-")
        pos_count_array_m6Am.append(str(read.query_name).split("_")[-1] + read.query_sequence)   
        pos_count_array_m6Am.append(str(read.query_sequence))                    
    elif read.query_sequence[39] == 'C' and chr_loc(f'chr{i+1}', read.reference_end-39, fasta) == 'T':
        if i == 22:
            pos_count_array_Am.append("chrX")
        if i == 23:
            pos_count_array_Am.append("chrY")
        else:
            pos_count_array_Am.append(f'chr{i+1}')
        pos_count_array_Am.append(read.reference_end-1)
        pos_count_array_Am.append(read.reference_end)
        pos_count_array_Am.append("-")
        pos_count_array_Am.append(str(read.query_name).split("_")[-1] + read.query_sequence)
        pos_count_array_Am.append(str(read.query_sequence))   
    else:
        if i == 22:
            pos_count_array.append("chrX")
        if i == 23:
            pos_count_array.append("chrY")
        else:
            pos_count_array.append(f'chr{i+1}')
        pos_count_array.append(read.reference_end-1)
        pos_count_array.append(read.reference_end)
        pos_count_array.append("-")
        pos_count_array.append(str(read.query_name).split("_")[-1] + read.query_sequence)
        pos_count_array.append(str(read.query_sequence))   

In [13]:
# Removes reads in which base pair mismatches occur at the first three positions for cap2 and the first two positions for cap1

pos_count1 = []
pos_count1_m6Am = []
pos_count1_Am = []
pos_count2 = []
pos_count2_m6Am = []
pos_count2_Am = []

for i in tqdm(range(24)):
    for j in range(len(reads1[i])):
        if reads1[i][j].is_forward==True:
            if reads1[i][j].query_sequence[0] == chr_loc(f'chr{i+1}', reads1[i][j].reference_end-39, fasta) or (reads1[i][j].query_sequence[0] == 'G' and chr_loc(f'chr{i+1}', reads1[i][j].reference_end-39, fasta) == 'A'):
                if reads1[i][j].query_sequence[1] == chr_loc(f'chr{i+1}', reads1[i][j].reference_end-38, fasta) or (reads1[i][j].query_sequence[1] == 'G' and chr_loc(f'chr{i+1}', reads1[i][j].reference_end-38, fasta) == 'A'): 
                    add_read_info_fwd(reads1[i][j], pos_count1_m6Am, pos_count1_Am, pos_count1)
                    
        if reads1[i][j].is_forward==False:
            if reads1[i][j].query_sequence[39] == chr_loc(f'chr{i+1}', reads1[i][j].reference_end, fasta) or (reads1[i][j].query_sequence[39] == 'C' and chr_loc(f'chr{i+1}', reads1[i][j].reference_end, fasta) == 'T'):
                if reads1[i][j].query_sequence[38] == chr_loc(f'chr{i+1}', reads1[i][j].reference_end-1, fasta) or (reads1[i][j].query_sequence[38] == 'C' and chr_loc(f'chr{i+1}', reads1[i][j].reference_end-1, fasta) == 'T'): 
                    add_read_info_rev(reads1[i][j], pos_count1_m6Am, pos_count1_Am, pos_count1)

    for j in range(len(reads2[i])):   
        if reads2[i][j].is_forward==True:
            if reads2[i][j].query_sequence[0] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end-39, fasta) or (reads2[i][j].query_sequence[0] == 'G' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-39, fasta) == 'A'):
                if reads2[i][j].query_sequence[1] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end-38, fasta) or (reads2[i][j].query_sequence[1] == 'G' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-38, fasta) == 'A'):
                    if reads2[i][j].query_sequence[2] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end-37, fasta) or (reads2[i][j].query_sequence[2] == 'G' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-37, fasta) == 'A'):
                        add_read_info_fwd(reads2[i][j], pos_count2_m6Am, pos_count2_Am, pos_count2)
        if reads2[i][j].is_forward==False:
            if reads2[i][j].query_sequence[39] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end, fasta) or (reads2[i][j].query_sequence[39] == 'C' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end, fasta) == 'T'):
                if reads2[i][j].query_sequence[38] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end-1, fasta) or (reads2[i][j].query_sequence[38] == 'C' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-1, fasta) == 'T'):
                    if reads2[i][j].query_sequence[37] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end-2, fasta) or (reads2[i][j].query_sequence[37] == 'C' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-2, fasta) == 'T'):
                        add_read_info_rev(reads2[i][j], pos_count2_m6Am, pos_count2_Am, pos_count2)

100%|███████████████████████████████████████████| 24/24 [00:04<00:00,  4.91it/s]


In [14]:
cap1Am = np.array(pos_count1_Am, dtype = 'object')
cap1m6Am = np.array(pos_count1_m6Am, dtype = 'object')

cap2Am = np.array(pos_count2_Am, dtype = 'object')
cap2m6Am = np.array(pos_count2_m6Am, dtype = 'object')
      
cap1Am = cap1Am.reshape((int(cap1Am.size/6), 6))
cap2Am = cap2Am.reshape((int(cap2Am.size/6), 6))

cap1m6Am = cap1m6Am.reshape((int(cap1m6Am.size/6), 6))
cap2m6Am = cap2m6Am.reshape((int(cap2m6Am.size/6), 6))

In [28]:
np.save("cap1_Am_nm.npy", cap1Am)
np.save("cap2_Am_nm.npy", cap2Am)
np.save("cap1_m6Am_nm.npy", cap1m6Am)
np.save("cap2_m6Am_nm.npy", cap2m6Am)